# Curve number assignments

In [19]:
import os
import numpy as np
import pandas as pd
import gdal

%run plot_and_table_functions.py

lu_lookup_fname  = '../std_input/Landuse_lookup_CDL.txt'
irr_lookup_fname = '../std_input/Irrigation_lookup_CDL.txt'
lu_grid_fname    = '../input/landuse.asc'

In [20]:
lu_lookup  = pd.read_table( lu_lookup_fname )
irr_lookup = pd.read_table( irr_lookup_fname )
lu_grid    = gdal.Open( lu_grid_fname )

## Curve Number "aligner" Equations

It is generally a good idea to maintain some degree of structure between the curve numbers used for the various soils groups. In SWB, the soils groups are simply numbered, starting with 1, and can represent any arbitrary grouping of soils. It is convenient to assume that hydrologic soils groups are represented by table values of 1, 2, 3, and 4, respectively.

Several authors have published equations or plots that show the relationship between the curve number for B, C, and D soils given the value for an A soil. One implementation of these "curve number aligners" are defined as functions below.

In [61]:
def curve_number_aligner__B( CN_A ):
    """
    Convert a curve number value defined for hydrologic soil group 'A'
    to a curve number defined for hydrologic soil group 'B',
    from Table 5, Curve Number Hydrology, 2009, ASCE.
    """
    return( 0.622 * CN_A + 37.8 )

def curve_number_aligner__C( CN_A ):
    """
    Convert a curve number value defined for hydrologic soil group 'A'
    to a curve number defined for hydrologic soil group 'C'
    from Table 5, Curve Number Hydrology, 2009, ASCE.    
    """
    return( 0.411 * CN_A + 58.9 )

def curve_number_aligner__D( CN_A ):
    """
    Convert a curve number value defined for hydrologic soil group 'A'
    to a curve number defined for hydrologic soil group 'D'
    from Table 5, Curve Number Hydrology, 2009, ASCE.    
    """
    return( 0.328 * CN_A + 67.2 )

In this example, the hydrologic soils group 'A' is listed in the lookup table as soil group '1'; curve numbers are defined in the landuse lookup tables for groups A, B, C, and D as CN_1, CN_2, CN_3, and CN_4, respectively.

In [64]:
lu_lookup[['LU_Code','Description','CN_1']].ix[2:7]

,LU_Code,Description,CN_1
2,5,Soybeans,67
3,6,Sunflowers,61
4,12,Sweet Corn,65
5,21,Barley,30
6,23,Spring Wheat,30
7,24,Winter Wheat,30


### Calculate curve numbers for B, C, and D soils given curve numbers for A soils.

In [65]:
lu_lookup['CN_2'] = curve_number_aligner__B( lu_lookup[['CN_1']] )
lu_lookup['CN_3'] = curve_number_aligner__C( lu_lookup[['CN_1']] )
lu_lookup['CN_4'] = curve_number_aligner__D( lu_lookup[['CN_1']] )
lu_lookup[['LU_Code','Description','CN_1','CN_2','CN_3','CN_4']].ix[2:7]

,LU_Code,Description,CN_1,CN_2,CN_3,CN_4
2,5,Soybeans,67,79.474,86.437,89.176
3,6,Sunflowers,61,75.742,83.971,87.208
4,12,Sweet Corn,65,78.230,85.615,88.520
5,21,Barley,30,56.460,71.230,77.040
6,23,Spring Wheat,30,56.460,71.230,77.040
7,24,Winter Wheat,30,56.460,71.230,77.040


### Write modified table back to disk

In [63]:
lu_lookup.to_csv( '../std_input/landuse_modified.txt', sep='\t' )